In [1]:
"""
@author: joey
@time: 2020/06/19
"""


'\n@author: joey\n@time: 2020/06/19\n'

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='ticks',context='poster')
SEED = 222
np.random.seed(SEED)
from joblib import Parallel, delayed

In [8]:

filepath = '/Users/joey/Documents/PycharmProjects/mywork/decision tree/science_federal_giving.csv'
df = pd.read_csv(filepath,nrows=3000)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def get_tarin_test(test_size=0.95):
    """split data into train and test sets"""
    df1 = df.drop(df[df.cand_pty_affiliation.isin(['IND','LIB'])].index)
    y = 1 * (df1.cand_pty_affiliation == 'REP') # 转换类型为0和1
    X = df1.drop(['cand_pty_affiliation','cand_office_district'], axis=1)
    X = X.dropna(axis=1)
    X = pd.get_dummies(X, sparse=True) # 字母属性转换为数字
    X.drop(X.columns[X.std() == 0], axis=1, inplace=True)
    print(y.shape,X.shape)
    return train_test_split(X, y, test_size=test_size, random_state=SEED)
xtrain, xtest, ytrain, ytest = get_tarin_test()

(2983,) (2983, 5248)


In [10]:
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.kernel_approximation import Nystroem, RBFSampler
from sklearn.pipeline import make_pipeline

def get_models():
    """Generate a library of base learners"""
    nb = GaussianNB()
    svc = SVC(C=100, probability=True)
    knn = KNeighborsClassifier(n_neighbors=3)
    lr = LogisticRegression(C=100, random_state=SEED, solver='liblinear', max_iter=3000)
    nn = MLPClassifier((80, 10), early_stopping=False, random_state=SEED)
    gb = GradientBoostingClassifier(n_estimators=100, random_state=SEED)
    rf = RandomForestClassifier(n_estimators=10, max_features=3, random_state=SEED)

    models = {'svm': svc, 'knn': knn, 'naive bayes': nb, 'mlp-nn':nn, 'random forest': rf, 'gbm': gb, 'logistic': lr}
    print('-'* 10)
    return models


In [ ]:
def train_predict(models_list):
    """Fit models in list on training set and return preds"""
    P = np.zeros((ytest.shape[0], len(models_list)))
    P = pd.DataFrame(P)

    print('Fitting models')
    cols = list()

    Parallel(n_jobs=8)(delayed())

    for i, (name, m) in enumerate(models_list.items()):
        print('%s...'% name, end=' ', flush=False)
        m.fit(xtrain, ytrain)
        P.iloc[:,i] = m.predict_proba(xtest)[:,1]
        cols.append(name)
        print('done')
    P.columns = cols
    print('done!')
    print('-'* 10)
    return P

In [ ]:
def score_models(P, ytest):
    """Score model in prediction DF"""
    print('Scoring models')
    for m in P.columns:
        score = roc_auc_score(ytest, P.loc[:,m])
        print('%s score is %.3f' % (m, score))
    print('done!')
    print('-'* 10)

In [ ]:
models_list = get_models()
P = train_predict(models_list)
score_models(P,ytest)
